In [162]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
py.init_notebook_mode(connected=True)
%matplotlib inline
import quandl
quandl.ApiConfig.api_key = "nUqyP3u7T-rY9U8sL92P"
import numpy as np
import pandas as pd
# import regular libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
import pymongo
import requests
from pandas.io.json import json_normalize
from statistics import stdev 
import json
import math

# from SciPy packages
from statsmodels.tsa.stattools import adfuller # adfuller test
from statsmodels.graphics.tsaplots import plot_acf # autocorellation plot
from statsmodels.graphics.tsaplots import plot_pacf # partial autocorellation plot

# math function
from math import sqrt

# evaluation metrics
from sklearn.metrics import mean_squared_error

# statistics models
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# libraries to filter warnings in some algorithms
import warnings
warnings.filterwarnings("ignore")

import json
import requests

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
%matplotlib inline
# import regular libraries 
from datetime import datetime
import time
import pymongo
import requests
from pandas.io.json import json_normalize
from statistics import stdev 
import json

# from SciPy packages
from statsmodels.tsa.stattools import adfuller # adfuller test
from statsmodels.graphics.tsaplots import plot_acf # autocorellation plot
from statsmodels.graphics.tsaplots import plot_pacf # partial autocorellation plot

# math function
from math import sqrt

# evaluation metrics
from sklearn.metrics import mean_squared_error

# statistics models
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# libraries to filter warnings in some algorithms
import warnings
warnings.filterwarnings("ignore")

# import pandas_datareader.data as web
from plotly.tools import mpl_to_plotly


In [163]:
myclient = pymongo.MongoClient(host = '139.162.167.213:27017',username="mongo-naascap",password="n44$c4p")
cw = "?api_key=XRSFO6KHEQC87X8UOYZJ"

ts='6h'
market='coinbase-pro'
quote='btc'



def boxAsset(t,c):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    l=[word.split("_")[2] for word in bn if(word.startswith(t+"_"+c))]
    l.sort()
    return l

def getAssetClose(t,c,a):
    cbp=myclient["Spot"]
    db=cbp[t+"_"+c+"_"+a]
    db=pd.DataFrame(list(db.find({},{"_id":1,"close":1}).sort("_id",-1)))
    db.rename(columns={'close':a},inplace=True)
    db.index=[(datetime.fromtimestamp(i)) for i in db._id]
    return db[a]


def getMarketCloses(ts):
    assets=boxAsset(ts,market)
    closes=[getAssetClose(ts,market,asset) for asset in assets]
    data = pd.concat(closes,join='outer',axis=1,sort=False)
    date_format = "%d/%m/%Y"
    date = datetime.strptime('01/01/2020', date_format)
    data=data[[date<=dates for dates in data.index]]
    return data


def getColumns(TT):
    df=pd.DataFrame([TT.columns,TT.columns]).T.set_axis(['id', 'name'], axis=1, inplace=False)
    L=[0]*(len(df))
    for i in (range(len(df))):
        L[i]={'id':df["id"][i],'name':df["name"][i]}
    return L
 
def listToDropDown(l):
    df=pd.DataFrame([l,l]).T.set_axis(['Value', 'Label'], axis=1, inplace=False)
    df=df.sort_values("Value").reset_index()
    L=[0]*len(df)
    for i in range(len(df)):
        L[i]={'label':df["Label"][i],'value':df["Value"][i]}
    return L




def TableBaseFromSymbol():
    URL="https://api.cryptowat.ch/pairs"
    res = requests.get(URL + cw)
    T=json_normalize(json.loads(res.content.decode('utf-8'))["result"])
    T=T.loc[:,["symbol","base.symbol","quote.symbol"]]
    return T
T = TableBaseFromSymbol()
def BaseFromSymbol(T,sym):
    if(sym in list(T.symbol)):
        return T["base.symbol"][T.symbol==sym].iloc[0]

def QuoteFromSymbol(T,sym):
    if(sym in list(T.symbol)):
        return T["quote.symbol"][T.symbol==sym].iloc[0]

def getAllAssetCloseByTime(t):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    bn=[word for word in bn if(word.split("_")[1]== market and word.split("_")[0]==t)]
    df=pd.DataFrame()
    l2=[]
    l=list(set([word.split("_")[2] for word in bn]))
    TT=list(set([BaseFromSymbol(T,i) for i in l if(QuoteFromSymbol(T,i)) == quote]))
    for asset in TT:
        l1=[word for word in bn if(BaseFromSymbol(T,word.split("_")[2])==asset and QuoteFromSymbol(T,word.split("_")[2]) == quote)]
        rr=[pd.DataFrame(list(cbp[i].find({},{ "_id": 1, "close": 1})),index=pd.DataFrame(list(cbp[i].find({},{ "_id": 1})))._id) for i in l1]
        data = pd.concat([i["close"] for i in rr],join='outer',axis=1,sort=False)
        l2.append(data.mean(axis=1))

    df=pd.concat(l2,axis=1)
    df.columns=(TT)
    df.index=[(datetime.fromtimestamp(i)) for i in df.index]
    df = df.fillna(method = "ffill")
    
    return df

def getAllAssetVolumeByTime(t):
    cbp=myclient["Spot"]
    bn=cbp.collection_names()
    bn=[word for word in bn if(word.split("_")[1]== market and word.split("_")[0]==t)]
    df=pd.DataFrame()
    l2=[]
    l=list(set([word.split("_")[2] for word in bn]))
    TT=list(set([BaseFromSymbol(T,i) for i in l if(QuoteFromSymbol(T,i)) == quote]))
    for asset in TT:
        l1=[word for word in bn if(BaseFromSymbol(T,word.split("_")[2])==asset and QuoteFromSymbol(T,word.split("_")[2]) == quote)]
        rr=[pd.DataFrame(list(cbp[i].find({},{ "_id": 1, "volume_quote": 1})),index=pd.DataFrame(list(cbp[i].find({},{ "_id": 1})))._id) for i in l1]
        data = pd.concat([i["volume_quote"] for i in rr],join='outer',axis=1,sort=False)
        l2.append(data.mean(axis=1))

    df=pd.concat(l2,axis=1)
    df.columns=(TT)
    df.index=[(datetime.fromtimestamp(i)) for i in df.index]
    df = df.fillna(0)
    
    return df


In [164]:
market='coinbase-pro' #'kraken'
quote='usd' #'btc'
ts='1d' # '5m' '2h' '6h' '12h' '1d'
df=getAllAssetCloseByTime(ts)
df1=getAllAssetVolumeByTime('1d') 

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
prices= df.fillna(df.mean())


In [ ]:
prices.head()

In [ ]:
data= df.tail(6)

In [ ]:
data.columns

In [ ]:
data= data.drop(columns=['zrx', 'mkr', 'oxt', 'band', 'omg', 'algo', 'comp', 'atom', 'xrp',
       'knc', 'xtz', 'dai'])

In [ ]:
def get_Close_data(df1,ch):
    df=df1[[ch]]
    x=len(df)%7 
    df=df.drop(df.index[0:x])
    return df

In [ ]:
df= get_Close_data(df,'btc')

Modern Portfolio

In [ ]:
prices.head()

In [ ]:
#list of stocks in portfolio
stocks = ['nmr', 'zrx', 'link', 'bch', 'etc', 'btc', 'dai', 'dash', 'xrp', 'band',
       'comp', 'rep', 'knc', 'omg', 'algo', 'mkr', 'atom', 'xlm', 'oxt', 'ltc',
       'xtz', 'eth', 'eos']

#download daily price data for each of the stocks in the portfolio
#data = web.DataReader(stocks,data_source='yahoo',start='01/01/2010')['Adj Close']

prices.sort_index(inplace=True)

#convert daily stock prices into daily returns
returns = prices.pct_change()
returns.replace([np.inf, -np.inf], np.nan, inplace=True)
returns= returns.fillna(returns.mean())

#calculate mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()

#set number of runs of random portfolio weights
num_portfolios = 25000

#set up array to hold results
#We have increased the size of the array to hold the weight values for each stock
results = np.zeros((4+len(stocks)-1,num_portfolios))

for i in range(num_portfolios):
    #select random weights for portfolio holdings
    weights = np.array(np.random.random(23))
    #rebalance weights to sum to 1
    weights /= np.sum(weights)
    
    #calculate portfolio return and volatility
    portfolio_return = np.sum(mean_daily_returns * weights) * 252
    portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252)
    
    #store results in results array
    results[0,i] = portfolio_return
    results[1,i] = portfolio_std_dev
    #store Sharpe Ratio (return / volatility) - risk free rate element excluded for simplicity
    results[2,i] = results[0,i] / results[1,i]
    #iterate through the weight vector and add data to results array
    for j in range(len(weights)):
        results[j+3,i] = weights[j]

#convert results array to Pandas DataFrame
results_frame = pd.DataFrame(results.T,columns=['ret','stdev','sharpe',stocks[0],stocks[1],stocks[2],stocks[3],stocks[4],stocks[5],stocks[6],stocks[7],
                                                stocks[8],stocks[9],stocks[10],stocks[11],stocks[12],stocks[13],stocks[14],stocks[15],stocks[16],stocks[17],stocks[18],stocks[19],stocks[20],stocks[21],
                                                stocks[22]])

#locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
#locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

#create scatter plot coloured by Sharpe Ratio
portfolio=plt.figure(figsize=[15,7])
plt.scatter(results_frame.stdev,results_frame.ret,c=results_frame.sharpe,cmap='RdYlBu')
plt.xlabel('Volatility')
plt.ylabel('Returns')
plt.colorbar()
#plot red star to highlight position of portfolio with highest Sharpe Ratio
plt.scatter(max_sharpe_port[1],max_sharpe_port[0],marker=(5,1,0),color='r',s=1000)
#plot green star to highlight position of minimum variance portfolio
plt.scatter(min_vol_port[1],min_vol_port[0],marker=(5,1,0),color='g',s=1000)

In [165]:
results_frame.head()

,ret,stdev,sharpe,nmr,zrx,link,bch,etc,btc,dai,...,omg,algo,mkr,atom,xlm,oxt,ltc,xtz,eth,eos
0,0.129188,0.261835,0.493396,0.072275,0.031552,0.015587,0.014263,0.050388,0.005481,0.087237,...,0.037114,0.064711,0.051982,0.014865,0.018804,0.047813,0.074534,0.038997,0.054933,0.066615
1,0.148437,0.279217,0.531619,0.054951,0.044570,0.053808,0.041645,0.063626,0.023555,0.057554,...,0.010585,0.045730,0.062125,0.037854,0.016086,0.050870,0.061851,0.050572,0.055640,0.064365
2,0.154146,0.299296,0.515028,0.049441,0.085006,0.075581,0.032956,0.030809,0.019587,0.062713,...,0.062175,0.049294,0.018077,0.006428,0.024102,0.076630,0.011426,0.006754,0.020911,0.027285
3,0.167316,0.322993,0.518018,0.034640,0.071739,0.053758,0.064631,0.076696,0.066090,0.022540,...,0.027937,0.040475,0.017785,0.057913,0.031703,0.016193,0.074812,0.017271,0.049615,0.041812
4,0.170123,0.316383,0.537713,0.036801,0.019607,0.067942,0.053254,0.066597,0.060124,0.063321,...,0.037795,0.069465,0.017369,0.058923,0.052652,0.006227,0.055046,0.037779,0.015072,0.035398


In [171]:
max_sharpe_port*100

ret       18.963623
stdev     29.448822
sharpe    64.395184
nmr        8.278533
zrx        7.702616
link       5.326283
bch        2.105575
etc        8.635786
btc        4.103547
dai        4.234030
dash       2.844848
xrp        0.212668
band       0.360987
comp       1.759257
rep        1.338545
knc        5.004073
omg        1.918988
algo       7.656409
mkr        5.621094
atom       5.815747
xlm        6.682237
oxt        5.792134
ltc        0.073340
xtz        6.381802
eth        3.739629
eos        4.411872
Name: 15115, dtype: float64

In [ ]:
d=[go.Scatter(
    x=results_frame.stdev,
    y=results_frame.ret,
    mode="markers",
    marker=dict(
            color='LightSkyBlue'),
        opacity=0.6
    ),
   go.Scatter(
    x=[max_sharpe_port[1]],
    y=[max_sharpe_port[0]],
    mode="markers",
    name='highest variance portfolio',   
    marker=dict(
            color='Red',
            size=35)
    ),
   go.Scatter(
    x=[min_vol_port[1]],
    y=[min_vol_port[0]],
    name='minimum variance portfolio',   
    mode="markers",
    marker=dict(
            color='Green',
            size=35)
   )
]


In [ ]:
lay=go.Layout(
            xaxis={
                'title': 'Volatility',
               
            },
            yaxis={
                'title': 'Returns',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )

In [ ]:
modern_portfolio= go.Figure(data=d,layout=lay)

In [ ]:

def convert_portfolios(portfolios):
    ''' Takes in a cvxopt matrix of portfolios, returns a list of portfolios '''
    port_list = []
    for portfolio in portfolios:
        temp = np.array(portfolio).T
        port_list.append(temp[0].tolist())
        
    return port_list


def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))  
    
    #S is the covariance matrix,  
    # diagonal is the variance of each stock:
    #[ 1.00e+00 -3.00e-02 -3.19e-02 -2.72e-02]
    #[-3.00e-02  9.19e-01 -1.26e-02 -3.74e-02]
    #[-3.19e-02 -1.26e-02  9.64e-01  6.39e-03]
    #[-2.72e-02 -3.74e-02  6.39e-03  9.79e-01]
    
    
    pbar = opt.matrix(np.mean(returns, axis=1))     #pbar is the mean return of each stock

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] #returns a list of objects
                  for mu in mus]

    port_list = convert_portfolios(portfolios)
    
    
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios] #np.sqrt returns the stdev, not variance
    
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    #print m1 # result: [ 159.38531535   -3.32476303    0.4910851 ]
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x'] #Is this the tangency portfolio? X1 = slope from origin?  

    return np.asarray(wt), returns, risks, port_list


weights, returns, risks, portfolios = optimal_portfolio(return_vec)

In [ ]:
# Matplotlib only plot: 
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o') #risks and returns are just arrays of points along the frontier
plt.show()


# Interactive plot using mpld3:
def interactive_plot(risks, returns, portfolios):
    fig, ax = plt.subplots()
    ax.grid(True, alpha=0.3)


    labels = []
    for i in range(len(risks)):
        rounds = map(lambda x: round(x,3), portfolios[i])
        label = " Risk: " + str(risks[i]) + "<br>" + " Return: " + str(returns[i])  + "<br>" + " Portfolio Weights: " + str(rounds)
        labels.append(str(label))

    points = ax.plot(risks, returns, 'y-o', color='b',
                     mec='k', ms=15, mew=1, alpha=.6)

    ax.set_xlabel('standard deviation')
    ax.set_ylabel('return')
    ax.set_title('Efficient Frontier', size=20)

    tooltip = plugins.PointHTMLTooltip(points[0], labels,
                                   voffset=10, hoffset=10)
    plugins.connect(fig, tooltip)

    return mpld3.display()

    
interactive_plot(risks, returns, portfolios)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd

import mpld3
from mpld3 import plugins

In [ ]:
def covmean_portfolio(covariances, mean_returns):
    ''' Returns an optimal portfolio given a covariance matrix and matrix of mean returns '''
    n = len(mean_returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]


    S = opt.matrix(covariances)  # convert array to matrix  

    pbar = opt.matrix(mean_returns)  # convert array to matrix

    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    
    #Convert portfolios to list for output
    port_list = convert_portfolios(portfolios)
    
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    frontier_returns = [blas.dot(pbar, x) for x in portfolios]  #differ from those input, renamed
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios] #np.sqrt returns the stdev, not variance
    
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(frontier_returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    
    # CALCULATE THE OPTIMAL PORTFOLIO
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']   

    return np.asarray(wt), frontier_returns, risks, port_list

In [ ]:


weights, returns, risks, portfolios = covmean_portfolio(cov_matrix, mean_daily_returns)

# Matplotlib only plot: 
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o') #risks and returns are just arrays of points along the frontier
plt.show()

# Interactive mpld3 plot:
interactive_plot(risks, returns, portfolios)

In [ ]:
cov_matrix=cov_matrix.to_numpy()

In [ ]:
ly=go.Layout(
            xaxis={
                'title': 'Standard deviation',
               
            },
            yaxis={
                'title': 'Returns',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )

In [ ]:
 dss=[go.Scatter(
    x=risks,
    y=returns,
    mode="markers",
    marker=dict(
            color='Blue'),
        opacity=0.6
    )
]

In [ ]:
modern_portf2= go.Figure(data=dss,layout=ly)

In [ ]:
covariances = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]] 

mean_returns = [1.2,5.0,3.0,1.5]

weightss, returnss, riskss, portfolioss = covmean_portfolio(covariances, mean_returns)

# Matplotlib only plot: 
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(riskss, returnss, 'y-o') #risks and returns are just arrays of points along the frontier
plt.show()


In [ ]:

    ds=[go.Scatter(
    x=riskss,
    y=returnss,
    mode="markers",
    marker=dict(
            color='Red',
            size=10),
        opacity=0.6
    )
]

In [ ]:
modern_portf1= go.Figure(data=ds,layout=ly)

## Dash ##

In [ ]:
import dash
import dash_table
import datetime
import dash_core_components as dcc
import dash_html_components as html
from sklearn.linear_model import LinearRegression
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objs as go
import numpy
import math
from keras.models import Sequential
from keras.layers import Dense

df = pd.read_csv('BTCUSD_daily.csv')
dft = pd.read_csv('ETHUSD_daily.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([

    html.Div([

        # Title
        html.Div([
          html.H1(children='Crypto Currency Prediction')
        ], style={'textAlign': 'center'}),

        # Drop down and data table
        html.Div([
            html.H3(children='Crypto Currency Type'),

            dcc.Dropdown(
                id='table-dropdown',
                options=[
                    {'label': 'Bitcoin', 'value': 'BTC'},
                    {'label': 'Ethirium', 'value': 'ETH'}
                ],
                value='BTC'
            ),


            html.H3(children='Data Table'),

            dash_table.DataTable(
                id='table',
                data=df.to_dict('records'),
                columns=[{'id': c, 'name': c} for c in df.columns],
                fixed_rows={'headers': True, 'data': 0},
                style_cell={'width': '150px'}
            )],
        ),

        html.Div([
            html.H3(children='Data Graph'),
             dcc.Markdown('''
Another type of plot that is useful to summarize the distribution of observations is candlestick. We can see the disparity in data with years. The first 3 years have much smaller values, and are almost insignificant compared to the values in 2017 and 2018            '''),

            dcc.Graph(id='data-graph')
        ]),

        html.Div([
            html.H3(children='Candlestick'),
             dcc.Markdown('''
            Then a series of curves can be plotted to visualize the dataset and understand if the data answers to a particular behaviour. The first type of plot visualization for time series is the line plot, which is done below for the evolution of the price with time. It can be seen that the price increased substantially since 2017, which is the year where most people became aware of cryptocurrencies. 
            '''),

            dcc.Graph(id='data-candlestick')
        ]),


        html.Div([
            html.H2(children='Prediction using linear regression'),

            dcc.Markdown('''
            #### Why linear regression ?
            Linear regression is one of the simplest model and not the main method that will be used in this prediction.
            It is only to being used as a learning experience for the team members and possible others.
            '''),

            dcc.Graph(id='graph')
        ]),

        html.Div([
            html.H2(children='Time series prediction with deep learning'),

            dcc.Graph(id='graph-learn')
        ]),
                
         html.Div([
            html.H3(children='Modern Portfolio Optimization'),

    

    dcc.Graph(
        id='example-graph',
        figure=modern_portfolio,
        
    )
        
    
            
        ]),
            html.Div([
            html.H3(children='Markowitzs Method  '),
            dcc.Markdown('''
The concept of an efficient frontier for investment portfolios originated with Harry Markowitz's Modern Portfolio Theory. Markowitz's main insight was that you can minimize the risk you take for any level of return by diversifying a portfolio. The end result is a hyperbola, often called the Markowitz Bullet, that demonstrates that greater returns are associated with higher risk           '''),
    

    

    dcc.Graph(
        id='marko',
        figure=modern_portf1,
        
    )
        
    
            
        ]),
            html.Div([
            html.H4(children='Efficient Frontier '),

    

    dcc.Graph(
        id='markov',
        figure=modern_portf2,
        
    )
        
    
            
        ]),
        
       
        
     
        
        
        
        

    ]),
])






















@app.callback(Output('table', 'data'), [Input('table-dropdown', 'value')])
def update_rows(value):
    # dff = df[df['some-column'] == value]
    if value == 'BTC':
        return df.to_dict('records')
    elif value == 'ETH':
        return dft.to_dict('records')


@app.callback(Output('data-graph', 'figure'), [Input('table-dropdown', 'value')])
def update_data_graph(value_dropdown):

    if value_dropdown == 'BTC':
        df_graph = df
        text = 'Bitcoin'
    elif value_dropdown == 'ETH':
        df_graph = dft
        text = 'Ethirium'

    return {
        'data': [go.Scatter(
            mode='lines',
            x=df_graph['Date'],
            y=df_graph['Open'],
            text=text,
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_graph["Date"].iloc[0], df_graph["Date"].iloc[250], df_graph["Date"].iloc[500],
                             df_graph["Date"].iloc[750], df_graph["Date"].iloc[1000], df_graph["Date"].iloc[-1]],
                # 'tickformat': '%m/%y'
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


@app.callback(Output('data-candlestick', 'figure'), [Input('table-dropdown', 'value')])
def update_graph(value_dropdown):
    if value_dropdown == 'BTC':
        df_to_candlestick = df
    elif value_dropdown == 'ETH':
        df_to_candlestick = dft
    return {
        'data': [go.Candlestick(
            x=df_to_candlestick['Date'],
            open=df_to_candlestick['Open'],
            high=df_to_candlestick['High'],
            low=df_to_candlestick['Low'],
            close=df_to_candlestick['Close'],
        )],
    }


def datetime_to_float(d):
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    # total_seconds will be in decimals (millisecond precision)
    return total_seconds


@app.callback(Output('graph', 'figure'), [Input('table-dropdown', 'value')])
def update_graph(value_dropdown):
    if value_dropdown == 'BTC':
        df_to_graph = df
        df_graph = df
        text = 'Bitcoin'
        X = df.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
        Y = df.iloc[:, 4].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column
    elif value_dropdown == 'ETH':
        df_to_graph = dft
        df_graph = dft
        text = 'Ethirium'
        X = dft.iloc[:, 0].values.reshape(-1, 1)  # values converts it into a numpy array
        Y = dft.iloc[:, 4].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column

    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions

    df_for_graph = pd.DataFrame(Y_pred, columns=['Prediction'])

    return {
        'data': [go.Scatter(
            name='Linear Regression Prediction',
            mode='lines',
            x=df_to_graph['Date'],
            y=df_for_graph['Prediction'],
            text='Linear Regression Line',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        ),
            go.Scatter(
                name='Actual Data',
                mode='lines',
                x=df_graph['Date'],
                y=df_graph['Open'],
                text=text,
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'red'}
                })
        ],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_to_graph['Date'].iloc[0], df_to_graph["Date"].iloc[250], df_to_graph["Date"].iloc[500],
                             df_to_graph["Date"].iloc[750], df_to_graph["Date"].iloc[1000], df_to_graph["Date"].iloc[-1]
                             ],
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }



# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)


@app.callback(Output('graph-learn', 'figure'), [Input('table-dropdown', 'value')])
def update_graph_learn(value_dropdown):
    if value_dropdown == 'BTC':
        csv_name = 'Gemini_BTCUSD_daily.csv'
        df_to_learn = df
    elif value_dropdown == 'ETH':
        csv_name = 'Gemini_ETHUSD_daily.csv'
        df_to_learn = dft

    # fix random seed for reproducibility
    numpy.random.seed(7)
    # load the dataset
    dataframe = pd.read_csv(csv_name, usecols=[6], engine='python')
    dataset = dataframe.values
    dataset = dataset.astype('float32')

    # split into train and test sets
    train_size = int(len(dataset) * 0.67)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

    # reshape dataset
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    # create and fit Multilayer Perceptron model
    model = Sequential()
    model.add(Dense(1, input_dim=look_back, activation='relu'))
    # model.add(Dense(8, activation='relu'))
    # model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=60, batch_size=2, verbose=2)

    # Estimate model performance
    trainScore = model.evaluate(trainX, trainY, verbose=0)
    print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
    testScore = model.evaluate(testX, testY, verbose=0)
    print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

    # generate predictions for training
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # shift train predictions for plotting
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict) + look_back, :] = trainPredict

    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict) + (look_back * 2) + 1:len(dataset) - 1, :] = testPredict

    dataset = pd.DataFrame(dataset, columns=['Data'])
    trainPredictPlot = pd.DataFrame(trainPredictPlot, columns=['Train'])
    testPredictPlot = pd.DataFrame(testPredictPlot, columns=['Test'])

    return {
        'data': [go.Scatter(
            name='Actual Data',
            mode='lines',
            x=df_to_learn['Date'],
            y=dataset['Data'],
            text='Actual Data',
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'blue'}
            }
        ),
            go.Scatter(
                name='Training',
                mode='lines',
                x=df_to_learn['Date'],
                y=trainPredictPlot['Train'],
                text='Training',
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'green'}
                }),

            go.Scatter(
                name='Test',
                mode='lines',
                x=df_to_learn['Date'],
                y=testPredictPlot['Test'],
                text='Test',
                marker={
                    'size': 15,
                    'opacity': 0.5,
                    'line': {'width': 0.5, 'color': 'red'}
                })
        ],
        'layout': go.Layout(
            xaxis={
                'title': 'Date',
                'tickvals': [df_to_learn['Date'].iloc[0], df_to_learn["Date"].iloc[250], df_to_learn["Date"].iloc[500],
                             df_to_learn["Date"].iloc[750], df_to_learn["Date"].iloc[1000], df_to_learn["Date"].iloc[-1]
                             ],
            },
            yaxis={
                'title': 'Price : USD',
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


In [ ]:
app.run_server(debug=False)